# Final Project Baseline: House Prices- Advanced Regression Techniques
### Author: Radia Abdul Wahab, Parham Motameni, Jun Qian
### Date: Fall 2021
### Course: w207 Machine Learning



# 

In [9]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import libraries.
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [10]:
# lock the seed to have repeatable results
random.seed(100)

In [11]:
# load datatsets and labels
X_train, X_test = pd.read_csv('X_train.csv'),  pd.read_csv('X_test.csv')
y_train, y_test=  pd.read_csv('y_train.csv'),  pd.read_csv('y_test.csv')

In [12]:
#  there are 12 MSSub Classes
pd.unique(X_train['MSSubClass']).shape

(15,)

## Fixup some features (pre-split)
TODO do all of these pre-split

### total number of bath = number of full + (number of half/2)

In [13]:
# Set half-bath to half of its value
bath_props = ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath']
X_train[['BsmtHalfBath', 'HalfBath']] = X_train[['BsmtHalfBath', 'HalfBath']].apply(lambda x: x/2, axis=1)
X_train['total_bath'] =  X_train[bath_props].apply(np.sum, axis=1)

### Age of house at sale time, remodlled age at sale time

In [14]:
X_train['Age'] = X_train['YrSold']-X_train['YearBuilt']
X_train['RemodAge'] = X_train['YrSold']-X_train['YearRemodAdd']
X_train[['YearBuilt','YearRemodAdd','YrSold', 'Age', 'RemodAge' ]].head()

,YearBuilt,YearRemodAdd,YrSold,Age,RemodAge
0,1940,1950,2007,67,57
1,2001,2002,2009,8,7
2,2002,2002,2006,4,4
3,1965,1965,2009,44,44
4,1995,1996,2007,12,11


## Selecting a set of feature to work on 
These features are selected based on analysis and impact of them on the price

In [15]:
list_of_features = ['MSSubClass', 'MSZoning', 'LotArea', 'Utilities', \
    'Neighborhood', 'OverallQual', 'Age', 'RemodAge', \
        'TotRmsAbvGrd', 'KitchenQual', 'YrSold', 'MoSold', 'GrLivArea', 'total_bath']
working_set = X_train[list_of_features]
working_set.head()

,MSSubClass,MSZoning,LotArea,Utilities,Neighborhood,OverallQual,Age,RemodAge,TotRmsAbvGrd,KitchenQual,YrSold,MoSold,GrLivArea,total_bath
0,50,RL,8160,AllPub,Edwards,5,67,57,7,TA,2007,4,1134,2.5
1,20,RL,16285,AllPub,CollgCr,7,8,7,6,Gd,2009,6,1430,2.0
2,60,RL,13450,AllPub,CollgCr,7,4,4,8,Gd,2006,6,1861,3.5
3,20,RL,9100,AllPub,NAmes,5,44,44,5,TA,2009,7,925,1.5
4,20,RL,7875,AllPub,Gilbert,7,12,11,6,TA,2007,6,1253,2.0


In [19]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.compose import make_column_transformer

# TODO use the X_train instead of temp k
mine_t = X_train[['MSSubClass', 'MSZoning', 'Utilities', 'Neighborhood', 'KitchenQual',  \
    'LotArea', 'Age', 'RemodAge', 'TotRmsAbvGrd', 'GrLivArea', 'total_bath']]

index_of_encoded_cols = [mine_t.columns.get_loc(col) for col in ['MSSubClass', 'MSZoning', 'Utilities', 'Neighborhood',  'KitchenQual']]
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), index_of_encoded_cols)], remainder='passthrough')
mine_t = ct.fit_transform(mine_t)
# print(mine_t.shape)
# print(mine_t)

## Training the Multiple Linear Regression model on the Training set

In [20]:
regressor = LinearRegression()
regressor.fit(mine_t, y_train)

LinearRegression()

## Predicting the Test set results for Multiple Linear Regression model 

In [21]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

ValueError: could not convert string to float: 'RL'